In [214]:
from bs4 import BeautifulSoup, Comment
import requests
import regex as re
import xml
from selenium import webdriver
import pandas as pd
import random
import time

In [215]:
website = ["https://www.kasikornbank.com/en/rate/pages/foreign-exchange.aspx",
"https://www.scb.co.th/th/personal-banking/foreign-exchange-rates.html",
"https://www.bangkokbank.com/th-TH/Personal/Other-Services/View-Rates/Foreign-Exchange-Rates",
"https://exchangerate.krungthai.com/#/counterRate",
"https://www.krungsri.com/th/exchangerate/Todayrates"]
#ธนาคารซื้อ #ธนาคารขาย

#Re function

In [217]:
def remove_tag(html_str):
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('!@', html_str)
    mylist = re.split(r'!@', text)
    clear_lst = []
    for clear_text in mylist:
        if (clear_text not in "\n") and (clear_text not in " ") and (clear_text not in ""):
            clear_lst.append(clear_text)
    return clear_lst
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html

In [7]:
# def re_find(tag_start,tag_end,text):
#     a = re.search(tag_start, text)
#     if a == None:
#         return ''
#     text = text[a.start():]
#     b = re.search(tag_end, text)
#     text = text[:b.end()]
#     #print(a.start(),b.start())
#     return text

In [8]:
# def re_find_all(tag_start,tag_end,text):
#     list_all = []
#     while(True):
#         a = re.search(tag_start, text)
#         #print(a.start(),b.end())
#         if(a == None):
#             #print("break")
#             return list_all
#         text = text[a.start():]
#         b = re.search(tag_end, text)
#         list_all.append(text[:b.end()])
#         text=text[b.end():]
#     return list_all

#start browser

In [218]:
def start_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('ignore-certificate-errors')
    # executable_path param is not needed if you updated PATH
    browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')
    return browser

def end_browser(browser):
    browser.quit()


#bank 
#special thanks #ppbasleng
https://github.com/ppbasleng

In [6]:
def kasikorn(url): 
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")

    #clean soup data
    # data = soup.find('div',{"id" : "divTableRate"}).find('tbody').find_all('tr')
    data = re.findall("<tr>(.+?)</tr>",re.findall("<tbody>(.+?)</tbody>",str(soup))[0])
    #re_find_all(r'<tr>',r'</tr>',re_find(r'<tbody>',r'</tbody>',str(soup)))
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        lst2 = lst[2:]
        lst_buy = [lst2[3],lst2[1],lst2[0],"-",lst2[2]]
        lst_sell = [lst2[5],lst2[4],lst2[4],lst2[4]]
        lst_rows.append([lst[0].replace(" ",""),lst_buy,lst_sell])
    
    return lst_rows
    #return soup

In [94]:
def scb(url): 
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    
    #clean soup data
    #data = re_find_all(r'<tr data-list-template="" style="">',r'</tr>',re_find(r'<table class="table-rate" data-table="">',r'</table>',str(soup)))

    remove = re.compile(r'\n')
    d = remove.sub('', str(soup))
    data = re.findall('<tr data-list-template="" style="">(.+?)</tr>',re.findall('<table class="table-rate" data-table="">(.+?)</table>',str(d))[0])

    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        lst2 = lst[2:]
        lst_buy = [lst2[5],lst2[3],lst2[4],"-",lst2[2]]
        lst_sell = [lst2[1],lst2[0],lst2[0],"-"]
        lst_rows.append([lst[0],lst_buy,lst_sell])
    return lst_rows
    #return soup
    

In [151]:
def bangkok(url):
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")

    #clean soup data
    # data = soup.find('table',{"class" : "table-primary table-foreign-exchange-rates blue"}).find('tbody').find_all('tr')
    #data = re_find_all(r'<tr>',r'</tr>',re_find(r'<tbody>',r'</tbody>',re_find(r'<table class="table-primary table-foreign-exchange-rates blue">',r'</table>',str(soup))))
    #data = re.findall
    remove = re.compile(r'\n')
    d = remove.sub('', str(soup))
    data = re.findall('<table class="table-primary table-foreign-exchange-rates blue">(.+?)</table>',str(d))
    data = re.findall('<tbody>(.+?)</tbody>',data[0])
    data = re.findall('<tr>(.+?)</tr>',data[0])

    lst_rows = []
    for index,value in enumerate(data):
        clear_lst = remove_tag(str(value))
        clear_lst = [x.replace(" ", "") for x in clear_lst]
        while(len(clear_lst) < 7):
            clear_lst.append('-')
        lst2 = clear_lst[2:]
        lst_buy = [lst2[0],'-','-',lst2[2],lst2[3]]
        lst_sell = [lst2[1],"-",lst2[4],lst2[4]]
        lst_rows.append([clear_lst[0],lst_buy,lst_sell])
    return lst_rows
    #return soup

In [189]:
def krungthai(url):
    browser.get(url)
    time.sleep(10)##########
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")

    #clean soup data
    #data = soup.find('table',{"id" : "cur-table-1"}).find('tbody').find_all('tr',{"class" : "ng-scope"})
    #data = re_find_all(r'<!-- ngIf: rate.isShow -->',r'<!-- end ngIf: rate.isShow -->',re_find(r'<tbody>',r'</tbody>',re_find(r'<table cellspacing="0" class="cur-table table table-bordered border-bottom-0" id="cur-table-1" style="table-layout: fixed; min-width: 688px;" width="100%">',r'</table>',str(soup))))
    remove = re.compile(r'\n')
    d = remove.sub('', str(soup))
    data = re.findall('<table cellspacing="0" class="cur-table table table-bordered border-bottom-0" id="cur-table-1" style="table-layout: fixed; min-width: 688px;" width="100%">(.+?)</table>',str(d))
    data = re.findall('<tbody>(.+?)</tbody>',str(data))
    data = re.findall('<!-- ngIf: rate.isShow -->(.+?)<!-- end ngIf: rate.isShow -->',str(data))
    
    lst_rows = []
    for index,value in enumerate(data):
        if index != 8 :
            clear_lst = remove_tag(str(value))
            clear_lst = [x.replace(" ", "") for x in clear_lst]
            clear_lst = [x.replace("Unq", "-") for x in clear_lst]
            lst2 = clear_lst[2:]
            lst_buy = [lst2[3],lst2[0],'-','-',lst2[1]]
            lst_sell = [lst2[4],lst2[2],'-','-']
            lst_rows.append([clear_lst[0],lst_buy,lst_sell])
    return lst_rows
    #return soup

In [211]:
def krungsri(url):
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")

    #clen soup data
    #data = soup.find('div',{"class" : "table-scroll"}).find('tbody').find_all('tr')
    #data = re_find_all(r'important">',r'</tr>',re_find(r'<tbody>',r'</tbody>',re_find(r'<div class="table-scroll">',r'</div>',str(soup))))
    remove = re.compile(r'\n')
    d = remove.sub('', str(soup))
    data = re.findall('<div class="table-scroll">(.+?)</div>',str(d))
    data = re.findall('<tbody>(.+?)</tbody>',str(data))
    data = re.findall('important">(.+?)</tr>',str(data))
    data = ["<"+d for d in data]
    
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        remove = re.compile(r'\s|\xa0|\*')
        lst = [remove.sub('', x) for x in lst]
        lst = [x.replace("UNQ.", "-") for x in lst]
        lst2 = lst[-5:]
        lst_buy = [lst2[0],lst2[2],'-','-',lst2[3]]
        lst_sell = [lst2[1],lst2[4],lst2[4],lst2[4]]
        lst_rows.append([lst[1],lst_buy,lst_sell])
    return lst_rows
    #return soup

#get currency

In [662]:
def get_currency(all_bank_data):
    currency_list = []
    for d in all_bank_data:
        for index,currency in enumerate(d):
            if index >= 3 :
                currency_list.append(currency[0])
            else:#USD
                if index == 0:
                    currency[0] = 'USD1'
                if index == 1:
                    currency[0] = 'USD5'
                if index == 2:
                    currency[0] = 'USD50'
                currency_list.append(currency[0])

    #return list(set([x for x in currency_list if currency_list.count(x) == len(all_bank_data)]))
    return list(set(currency_list))

#start browser

In [212]:
browser = start_browser()
kasi_data = kasikorn(website[0])
scb_data = scb(website[1])
bangkok_data = bangkok(website[2])
krungthai_data = krungthai(website[3])
krungsri_data = krungsri(website[4])
end_browser(browser)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14084\2120961943.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [125]:
f = open('text.txt', 'w')
with open('text.txt', 'w',encoding='utf-8') as f:
    f.write(str(soup))

In [444]:
# k_test = str(kasi_data)
# scb_test = str(scb_data)
# bangkok_test = str(bangkok_data)
# krungthai_test = str(krungthai_data)
# krungsri_test = str(krungsri_data)

In [572]:
# f = open('text.txt', 'w')
# with open('text.txt', 'w',encoding='utf-8') as f:
#     f.write(str(krungsri_test))

In [ ]:
# scb_test

In [667]:
len(krungthai_data)

25

In [668]:
len(kasi_data)

33

In [669]:
len(scb_data)

29

In [670]:
len(bangkok_data)

33

In [671]:
len(krungsri_data)

30

#test bank

In [ ]:
# my_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"}
# url = 'https://exchangerate.krungthai.com/'
# res = requests.get(url,headers=my_headers)
# soup = BeautifulSoup(res.text, 'html.parser')


#get_currency

In [710]:
all_bank_data = [kasi_data.copy(),scb_data.copy(),bangkok_data.copy(),krungthai_data.copy(),krungsri_data.copy()]


In [714]:
all_bank_data[4][0]

['USD1',
 ['32.42000', '33.18000', '-', '-', '33.28000'],
 ['33.80000', '33.78000', '33.78000', '33.78000']]

In [715]:
currency_list = get_currency(all_bank_data)

In [716]:
len(currency_list)

35

#pack all data to csv file

In [717]:

############## all currency without usd1,5-20,50-100
for currency in currency_list:
    ##########
    bank_name = {0 : 'ธนาคารกสิกรไทย',1 : 'ธนาคารไทยพาณิชย์',2 : 'ธนาคารกรุงเทพ',3 : 'ธนาคารกรุงไทย',4 : 'ธนาคารกรุงศรีอยุธยา'}

    bank = pd.DataFrame()
    bank = bank.reindex(columns=['ธนาคาร'])
    #bank.columns = pd.MultiIndex.from_product(bank.columns])

    buy_rate = ['ธนบัตร', 'ตั๋วเงิน', 'เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
    br = pd.DataFrame()
    br = br.reindex(columns=buy_rate)
    br.columns = pd.MultiIndex.from_product([["ราคาที่ธนาคารรับซื้อ"], br.columns])

    sell_rate = ['ธนบัตร', 'เช็คเดินทาง', 'ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
    sr = pd.DataFrame()
    sr = sr.reindex(columns=sell_rate)
    sr.columns = pd.MultiIndex.from_product([["ราคาที่ธนาคารขาย"], sr.columns])

    result = pd.concat([br,sr], axis=1, join='inner')
    result = pd.concat([bank,result], axis=1, join='inner')##########################
    

    for index,bank in enumerate(all_bank_data):
        data_list = []
        get_cur = False      
        for currency_data in bank:
            if currency_data[0] == currency:
                data_list = [bank_name[index]]+currency_data[1]+currency_data[2]
                result.loc[len(result)] = data_list
                get_cur = True
                break
        if get_cur == False:
            #print(bank_name[index],currency)
            result.loc[len(result)] = [bank_name[index]]+['-','-','-','-','-']+['-','-','-','-']
                

    result.to_csv('bank_buy_sell_rate/'+str(currency)+'.csv',index=False)

In [718]:
result

,ธนาคาร,"(ราคาที่ธนาคารรับซื้อ, ธนบัตร)","(ราคาที่ธนาคารรับซื้อ, ตั๋วเงิน)","(ราคาที่ธนาคารรับซื้อ, เช็คเดินทาง)","(ราคาที่ธนาคารรับซื้อ, ตั๋วแลกเงิน&ดราฟ)","(ราคาที่ธนาคารรับซื้อ, โอนเงินทางโทรเลข/โอนเงิน)","(ราคาที่ธนาคารขาย, ธนบัตร)","(ราคาที่ธนาคารขาย, เช็คเดินทาง)","(ราคาที่ธนาคารขาย, ตั๋วแลกเงิน&ดราฟ)","(ราคาที่ธนาคารขาย, โอนเงินทางโทรเลข/โอนเงิน)"
0,ธนาคารกสิกรไทย,7.72743,-,-,-,8.73333,9.61151,9.55964,9.55964,9.55964
1,ธนาคารไทยพาณิชย์,7.8,-,-,-,-,9.99,-,-,-
2,ธนาคารกรุงเทพ,7.99,-,-,-,-,9.17,-,-,-
3,ธนาคารกรุงไทย,7.57,-,-,-,8.65198,9.69,9.67476,-,-
4,ธนาคารกรุงศรีอยุธยา,7.58000,-,-,-,-,9.57000,-,-,-


In [33]:
# buy_rate = ['ธนบัตร', 'ตั๋วเงิน', 'เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
# br = pd.DataFrame()
# br = br.reindex(columns = buy_rate)

In [34]:
# sell_rate = ['ธนบัตร', 'เช็คเดินทาง', 'ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
# sr = pd.DataFrame()
# sr = sr.reindex(columns = sell_rate)

In [35]:
# br

In [36]:
# sr

In [37]:
# for i,d in enumerate(kasi_data):
#     #print(d[0]+d[1])
#     dd = d[1]
#     br.loc[len(br)] = dd

In [38]:
# br

In [39]:
#df.to_csv('thb-usd.csv',index=False)

In [40]:
# f = open('text.txt', 'w')
# with open('text.txt', 'w',encoding='utf-8') as f:
#     f.write(str(soup))

In [41]:
#soup